# Web Scraping
## Scraping our cost data
This notebook will be used to grab cost data from the desired website. From here we will conduct light data cleaning on the set in preparation for combining this with our main movie sets.
### Our cost data is coming from the following website [Here](https://www.the-numbers.com/movie/budgets/all)
Below we will be importing all necessary packages and ensuring when we preview our DataFrame we are able to see more data.

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = "{:.2f}".format

### We will be scraping cost data in order to pair this with our imdb data sets which were cleaned [Here](Data_Cleaning.ipnyb)

This function should return a series of lists with cost data that will be compiled.

In [126]:
aTags = []
dataClass = []
releaseDate = []
movie_Name = []
prodBudget = []
domesticGross = []
worldGross = []
movieIndex = []

def scrapeData(url):
    aTags = []
    dataClass = []
    releaseDate = []
    movie_Name = []
    prodBudget = []
    domesticGross = []
    worldGross = []
    movieIndex = []
    counter = 0
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    main = soup.find('div', id='page_filling_chart')
    center = main.find('center')
    table = center.find('table')
    tableBody = table.findAll('tr')
    for td in tableBody:
        data = td.findAll('td', class_='data')
        desc = td.findAll('td')
        for a in desc:
            tag = a.find('a', href=True)
            if tag is not None:
                aTags.append(tag.text.strip())
        for d in data:
            dataClass.append(d.text.strip())
    for i in list(range(0,len(aTags))):
        if i == 0:
            releaseDate.append(aTags[i])
        elif i%2 == 0:
            releaseDate.append(aTags[i])
        elif i%2 == 1:
            movie_Name.append(aTags[i])
    for i in dataClass:
        if counter == 0:
            movieIndex.append(i)
            counter += 1
        elif counter == 1:
            prodBudget.append(i)
            counter += 1
        elif counter == 2:
            domesticGross.append(i)
            counter += 1
        elif counter == 3:
            worldGross.append(i)
            counter = 0
    return movieIndex, releaseDate, movie_Name, prodBudget, domesticGross, worldGross
#     releaseDate = []
#     df = pd.DataFrame([movieIndex, movie_Name, releaseDate, prodBudget, 
#                        domesticGross, worldGross]).transpose()
#     df.columns = ['index', 'movieName', 
#                  'releaseDate', 'prodBudget',
#                  'domesticGross', 'worldGross']
    
# eventDate = [span.find('time').text.strip() for span in eventListing.findAll('span', style="display:none;")]

### This function fetches the next URL in the web page
It turns out all you need to do is change the last number by 100 for each page.

In [127]:
def enumerate_url(num):
    x = 101
    urlList = ['https://www.the-numbers.com/movie/budgets/all']
    for i in list(range(0,num)):
        if i == 0:
            urlList.append('https://www.the-numbers.com/movie/budgets/all/{}'.format(x))
        else:
            x += 100
            urlList.append('https://www.the-numbers.com/movie/budgets/all/{}'.format(x))
    return urlList       


### This function combines the first and second function.
Here we initialize the script which will be run on the-numbers.com and will grab all of my cost data and place this in a DataFrame. From this DataFrame we will export it to a CSV for further processing.

In [137]:
def scraper(number):
    releaseDateCompile = []
    movie_NameCompile = []
    prodBudgetCompile = []
    domesticGrossCompile = []
    worldGrossCompile = []
    movieIndexCompile = []
    urls = enumerate_url(number)
    for url in urls:
        movieIndex, releaseDate, movie_Name, prodBudget, domesticGross, worldGross = scrapeData(url)
        releaseDateCompile = releaseDateCompile + releaseDate
        movie_NameCompile = movie_NameCompile + movie_Name
        prodBudgetCompile = prodBudgetCompile + prodBudget
        domesticGrossCompile = domesticGrossCompile + domesticGross
        worldGrossCompile = worldGrossCompile + worldGross
        movieIndexCompile = movieIndexCompile + movieIndex
        time.sleep(1.5)
    df = pd.DataFrame([movieIndexCompile, movie_NameCompile, releaseDateCompile, prodBudgetCompile, 
                       domesticGrossCompile, worldGrossCompile]).transpose()
    df.columns = ['index', 'movieName', 
                 'releaseDate', 'prodBudget',
                 'domesticGross', 'worldGross']
    return df
dfBudgetInfo = scraper(200)

dfBudgetInfo.to_csv('budgetinfo.csv', index=False)

### This section will be used to clean the data scraped from the website.
We will be removing movies that have not been released yet, and movies which have 0 cost data.

In [3]:
df = pd.read_csv('budgetinfo.csv')

#### This section of cleaning will be getting our cost data into a more mutable format.

In [4]:
#remove $ from the data
df.prodBudget = [x[1:] for x in df.prodBudget]
df.domesticGross = [x[1:] for x in df.domesticGross]
df.worldGross = [x[1:] for x in df.worldGross]
#remove commas so we can convert into integers
df.prodBudget = df.prodBudget.str.replace(',', '')
df.domesticGross = df.domesticGross.str.replace(',', '')
df.worldGross = df.worldGross.str.replace(',', '')
#convert into integers
df.prodBudget = df.prodBudget.astype('int64', copy=False)
df.domesticGross = df.domesticGross.astype('int64', copy=False)
df.worldGross = df.worldGross.astype('int64', copy=False)

In [5]:
df.head()

,index,movieName,releaseDate,prodBudget,domesticGross,worldGross
0,1,Avengers: Endgame,"Apr 23, 2019",400000000,858373000,2797800564
1,2,Pirates of the Caribbean: On Stranger Tides,"May 20, 2011",379000000,241063875,1045663875
2,3,Avengers: Age of Ultron,"Apr 22, 2015",365000000,459005868,1403013963
3,4,Star Wars Ep. VII: The Force Awakens,"Dec 16, 2015",306000000,936662225,2068223624
4,5,Avengers: Infinity War,"Apr 25, 2018",300000000,678815482,2048359754


#### This section will be utilized for cleaning the dates and dropping and rows with 0 cost, or release dates that have not happened yet.

In [6]:
#change dates to numbers to convert column into date values
df.releaseDate = df.releaseDate.str.replace('Jan', '01')
df.releaseDate = df.releaseDate.str.replace('Feb', '02')
df.releaseDate = df.releaseDate.str.replace('Mar', '03')
df.releaseDate = df.releaseDate.str.replace('Apr', '04')
df.releaseDate = df.releaseDate.str.replace('May', '05')
df.releaseDate = df.releaseDate.str.replace('Jun', '06')
df.releaseDate = df.releaseDate.str.replace('Jul', '07')
df.releaseDate = df.releaseDate.str.replace('Aug', '08')
df.releaseDate = df.releaseDate.str.replace('Sep', '09')
df.releaseDate = df.releaseDate.str.replace('Oct', '10')
df.releaseDate = df.releaseDate.str.replace('Nov', '11')
df.releaseDate = df.releaseDate.str.replace('Dec', '12')
#convert spaces to - and , to no space
df.releaseDate = df.releaseDate.str.replace(' ', '/')
df.releaseDate = df.releaseDate.str.replace(',', '')
df.releaseDate = df.releaseDate.astype('str', copy=False)

df['releaseYear'] = df.releaseDate.str.slice(-4,10)

In [7]:
df.head()

,index,movieName,releaseDate,prodBudget,domesticGross,worldGross,releaseYear
0,1,Avengers: Endgame,04/23/2019,400000000,858373000,2797800564,2019
1,2,Pirates of the Caribbean: On Stranger Tides,05/20/2011,379000000,241063875,1045663875,2011
2,3,Avengers: Age of Ultron,04/22/2015,365000000,459005868,1403013963,2015
3,4,Star Wars Ep. VII: The Force Awakens,12/16/2015,306000000,936662225,2068223624,2015
4,5,Avengers: Infinity War,04/25/2018,300000000,678815482,2048359754,2018


In [8]:
df['releaseYear'] = df.releaseYear.str.slice(-4,10)

df = df[df['releaseYear'] != 'nown']

df.releaseYear.unique()

array(['2019', '2011', '2015', '2018', '2007', '2017', '2012', '2016',
       '2010', '2009', '2013', '2014', '2020', '2006', '2008', '2005',
       '1997', '2004', '1999', '1995', '2003', '2001', '2002', '1998',
       '2000', '1991', '1994', '1996', '1993', '1992', '2021', '1988',
       '1990', '1989', '1978', '1981', '1984', '1982', '1985', '1980',
       '1963', '1987', '1986', '2022', '1983', '1979', '1977', '1970',
       '1969', '1976', '1965', '1962', '1964', '1959', '1966', '1974',
       '1956', '1975', '1973', '1960', '1967', '1968', '1971', '1951',
       '1972', '1961', '1946', '1944', '1953', '1954', '1957', '1952',
       '1930', '1939', '1925', '1950', '1948', '1958', '1943', '1940',
       '1945', '1947', '1938', '1927', '1949', '1955', '1936', '1937',
       '1941', '1942', '1933', '1935', '1931', '1916', '1929', '1934',
       '1915', '1920'], dtype=object)

In [9]:
df['releaseYear'] = df['releaseYear'].astype('str', copy=False).astype('int64', copy=False)

print(df.dtypes)

df = df[(df['releaseYear'] >= 2005) & (df['releaseYear'] <= 2020)]

index            object
movieName        object
releaseDate      object
prodBudget        int64
domesticGross     int64
worldGross        int64
releaseYear       int64
dtype: object


In [10]:
df.releaseYear.unique()

array([2019, 2011, 2015, 2018, 2007, 2017, 2012, 2016, 2010, 2009, 2013,
       2014, 2020, 2006, 2008, 2005])

In [11]:
len(df)

3513

In [26]:
df['estimateAdCost'] = df.apply(lambda x: x.prodBudget/3, axis=1)

In [27]:
df.head()

,index,movieName,releaseDate,prodBudget,domesticGross,worldGross,releaseYear,estimateAdCost,estimatedProfit,releaseMonth,releaseSeason
0,1,Avengers: Endgame,04/23/2019,400000000,858373000,2797800564,2019,133333333.33,2264467230.67,4,spring
1,2,Pirates of the Caribbean: On Stranger Tides,05/20/2011,379000000,241063875,1045663875,2011,126333333.33,540330541.67,5,spring
2,3,Avengers: Age of Ultron,04/22/2015,365000000,459005868,1403013963,2015,121666666.67,916347296.33,4,spring
3,4,Star Wars Ep. VII: The Force Awakens,12/16/2015,306000000,936662225,2068223624,2015,102000000.00,1660223624.00,12,winter
4,5,Avengers: Infinity War,04/25/2018,300000000,678815482,2048359754,2018,100000000.00,1648359754.00,4,spring


In [28]:
df['estimatedProfit'] = df.worldGross - (df.prodBudget + df.estimateAdCost)

df.head()

,index,movieName,releaseDate,prodBudget,domesticGross,worldGross,releaseYear,estimateAdCost,estimatedProfit,releaseMonth,releaseSeason
0,1,Avengers: Endgame,04/23/2019,400000000,858373000,2797800564,2019,133333333.33,2264467230.67,4,spring
1,2,Pirates of the Caribbean: On Stranger Tides,05/20/2011,379000000,241063875,1045663875,2011,126333333.33,540330541.67,5,spring
2,3,Avengers: Age of Ultron,04/22/2015,365000000,459005868,1403013963,2015,121666666.67,916347296.33,4,spring
3,4,Star Wars Ep. VII: The Force Awakens,12/16/2015,306000000,936662225,2068223624,2015,102000000.00,1660223624.00,12,winter
4,5,Avengers: Infinity War,04/25/2018,300000000,678815482,2048359754,2018,100000000.00,1648359754.00,4,spring


In [19]:
df['releaseMonth'] = df.releaseDate.str.slice(0,2)

df['releaseMonth'] = df.releaseMonth.astype('str', copy=False).astype('int32', copy=False)

print(df.dtypes)
# function to convert a list of month numbers to their season
# Function courtesy of Scott Johnson from Flatiron DS Course
def convert_num_seasons(months_list):
    num_season_dict = {1: 'winter', 2: 'winter', 3: 'spring',
                      4: 'spring', 5: 'spring', 6: 'summer',
                      7: 'summer', 8: 'summer', 9: 'fall',
                      10: 'fall', 11: 'fall', 12: 'winter'}
    converted = []
    for month in months_list:
        converted.append(num_season_dict.get(month))
    return converted

monthList = df['releaseMonth'].tolist()

seasonList = convert_num_seasons(monthList)

seasonList = np.array(seasonList)
# col_one_list = df['one'].tolist()

# col_one_arr = df['one'].to_numpy()

df.head()

index               object
movieName           object
releaseDate         object
prodBudget           int64
domesticGross        int64
worldGross           int64
releaseYear          int64
estimateAdCost     float64
estimatedProfit    float64
releaseMonth         int32
dtype: object


,index,movieName,releaseDate,prodBudget,domesticGross,worldGross,releaseYear,estimateAdCost,estimatedProfit,releaseMonth
0,1,Avengers: Endgame,04/23/2019,400000000,858373000,2797800564,2019,200000000.00,2197800564.00,4
1,2,Pirates of the Caribbean: On Stranger Tides,05/20/2011,379000000,241063875,1045663875,2011,189500000.00,477163875.00,5
2,3,Avengers: Age of Ultron,04/22/2015,365000000,459005868,1403013963,2015,182500000.00,855513963.00,4
3,4,Star Wars Ep. VII: The Force Awakens,12/16/2015,306000000,936662225,2068223624,2015,153000000.00,1609223624.00,12
4,5,Avengers: Infinity War,04/25/2018,300000000,678815482,2048359754,2018,150000000.00,1598359754.00,4


In [20]:
df['releaseSeason'] = seasonList

In [29]:
df.head()

,index,movieName,releaseDate,prodBudget,domesticGross,worldGross,releaseYear,estimateAdCost,estimatedProfit,releaseMonth,releaseSeason
0,1,Avengers: Endgame,04/23/2019,400000000,858373000,2797800564,2019,133333333.33,2264467230.67,4,spring
1,2,Pirates of the Caribbean: On Stranger Tides,05/20/2011,379000000,241063875,1045663875,2011,126333333.33,540330541.67,5,spring
2,3,Avengers: Age of Ultron,04/22/2015,365000000,459005868,1403013963,2015,121666666.67,916347296.33,4,spring
3,4,Star Wars Ep. VII: The Force Awakens,12/16/2015,306000000,936662225,2068223624,2015,102000000.00,1660223624.00,12,winter
4,5,Avengers: Infinity War,04/25/2018,300000000,678815482,2048359754,2018,100000000.00,1648359754.00,4,spring


In [22]:
df.to_csv('budgetinfo.csv', index=False)

### Scraping additional categorical data
Here we will grab some additional data from the same site.

In [151]:
def scrape_numbers(url, extension):
    index = []
    totalMovies = []
    avgBudget = []
    worldWide = []
    num_one = []
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html5lib')
    main = soup.find('div', id='main')
    content = soup.find('div', class_= "content", id=extension)
    center = content.find('center')
    table = center.find('table')
    rows = table.findAll('tr')
    for td in rows: 
        data = td.findAll('td', class_='data')
        indx = td.findAll('a', href=True)
        for i in indx:
            index.append(i.text.strip())
        for d in data:
            totalMovies.append(d.text.strip())
    for x in list(range(0,len(index))):
        if x % 2 == 1:
            num_one.append(index[x])
    avgBudget =totalMovies[1::3]
    del totalMovies[1::3]
    worldWide = totalMovies[1::2]
    del totalMovies[1::2]
    del index[1::2]
        
    return index, totalMovies, avgBudget, worldWide, num_one
desiredTabs = ['genre', 'mpaa_rating', 'production_method', 'creative_type', 'source']
url = 'https://www.the-numbers.com/movies/#tab=genre'
index = []
totalMovies = []
avgBudget = []
worldWide = []
num_one = []
for tab in desiredTabs:
    index, totalMovies, avgBudget, worldWide, num_one = scrape_numbers(url, tab)
    df = pd.DataFrame([index, totalMovies, avgBudget, worldWide, num_one]).transpose()
    df.columns = ['index', 'totalMovies', 'avgBudget', 'worldWide', 'num_one']
    df.to_csv('{}.csv'.format(tab), index=False)
# scrape_numbers('https://www.the-numbers.com/movies/#tab=genre', 'mpaa_rating')


In [154]:
df = pd.read_csv('genre.csv')
df.drop(15, inplace=True)
df.set_index('index')
df.to_csv('genre.csv', index=False)

In [155]:
df = pd.read_csv('mpaa_rating.csv')
df.set_index('index')
df.to_csv('mpaa_rating.csv', index=False)

In [156]:
df = pd.read_csv('production_method.csv')
df.set_index('index')
df.to_csv('production_method.csv', index=False)

In [157]:
df = pd.read_csv('creative_type.csv')
df.set_index('index')
df.to_csv('creative_type.csv',index=False)

In [158]:
df = pd.read_csv('source.csv')
df.drop(22, inplace=True)
df.drop(23, inplace=True)
df.set_index('index')
df.to_csv('source.csv',index=False)

In [159]:
df

,index,totalMovies,avgBudget,worldWide,num_one
0,Original Screenplay,"21,363","$25,460,842","$309,790,502,568",Avatar
1,Based on Fiction Book/Short…,"3,971","$39,837,852","$142,804,956,421",Jurassic World
2,Based on Comic/Graphic Novel,556,"$98,563,352","$62,505,364,926",Avengers: Endgame
3,Remake,490,"$47,224,118","$30,736,838,610",The Lion King
4,Based on Real Life Events,"6,411","$19,004,978","$29,416,960,859",Bohemian Rhapsody
5,Based on TV,432,"$54,678,580","$29,404,520,380",Transformers: Dark of the Moon
6,Based on Factual Book/Article,404,"$33,170,769","$13,621,682,365",Fast and Furious 6
7,Based on Folk Tale/Legend/F…,231,"$71,417,458","$10,868,339,514",Frozen
8,Spin-Off,83,"$98,300,000","$10,214,871,369",Minions
9,Based on Game,89,"$60,406,818","$6,498,089,704",Warcraft


In [160]:
def add_column(files):
    for f in files:
        df = pd.read_csv('{}.csv'.format(f))
        df['worldWide'] = df.worldWide.str.replace('$', '')
        df['worldWide'] = df.worldWide.str.replace(',','')
        df['worldWide'] = df.worldWide.astype('str', copy = False).astype('int64', copy=False)
        df['avgBudget'] = df.avgBudget.str.replace('$', '')
        df['avgBudget'] = df.avgBudget.str.replace(',','')
        df['avgBudget'] = df.avgBudget.astype('int32', copy=False)
        df['preAdRevenue'] = df.worldWide - df.avgBudget
        df.to_csv('{}.csv'.format(f), index=False)
add_column(desiredTabs)


In [161]:
df = pd.read_csv('genre.csv')

In [162]:
df

,index,totalMovies,avgBudget,worldWide,num_one,preAdRevenue
0,Adventure,"2,857",72069918,181368345526,Star Wars Ep. VII: The Force Awakens,181296275608
1,Action,"3,029",61899099,158166019677,Avengers: Endgame,158104120578
2,Drama,"11,876",19652562,107656670295,Bohemian Rhapsody,107637017733
3,Comedy,"6,215",22199268,91875302394,The Hangover Part II,91853103126
4,Thriller/Suspense,"3,189",28077430,53192263409,Titanic,53164185979
5,Horror,"2,666",14500554,29165003419,It,29150502865
6,Romantic Comedy,"1,488",24491702,25594982889,Mei Ren Yu,25570491187
7,Musical,458,30860780,14146588703,Frozen,14115727923
8,Documentary,"5,343",3157930,4949747737,Michael Jackson's This Is It,4946589807
9,Western,326,26831359,3905069532,Django Unchained,3878238173
